# Flajolet-Martin algorithm

We will implement the Flajolet Martin algorithm for counting the number of distinct elements in a stream or a very large data. Refer to the <a href="https://drive.google.com/file/d/18uY18kqGPMcW67bAjNiltTk6sSr6Wh2B/view?usp=sharing">slides</a> for the details. 

As in the case of bloom filter, we will use the MurmurHash 3 as our hash function, with different seeds. 

In [ ]:
import math
import numpy as np
import mmh3

## Outline

The algorithm uses several hash functions, each of which sends every element to a bitarray of size $nBits$. The hash functions are divided into $nGroup$ groups, each group having $nHash$ hash functions. 

Each hash function keeps track of the maximum tail length $R$ seen so far. The final estimate is obtained by combining the $2^R$ values from the different hash functions. 

<b>Note:</b> The MMDS book suggests taking average of each group and median of the averages. However, in my experiments (and in Wikipedia too), I found the other way works better: median of the groups and average of the medians. In this implementation, the two approaches are implemented as estimate1 and estimate2 respectively. 

In [ ]:
class FlajoletMartinEstimator(object):
    
    def __init__(self, nBits, nHash, nGroups): 
        self.nBits = nBits
        self.nHash = nHash
        self.nGroups = nGroups
        self.maxTailLength = np.zeros((self.nHash, self.nGroups))
        #print(self.maxTailLength)
        
    
    def tail_length(self, num_str):
        return len(num_str)-len(num_str.rstrip('0')) 
    
    
    def fmHash(self, item,i):
        return bin(mmh3.hash(item,i) % 2**self.nBits)
    
    
    def estimate1(self):
        # Average of 2^R within each group 
        # and median of the estimates from the groups
        averages = np.mean(2**self.maxTailLength, axis=0)
        # print('averages:', averages)
        return np.median(averages)
    
    def estimate2(self):
        # Median of 2^R within each group 
        # and average of the estimates from the groups
        medians = np.median(2**self.maxTailLength, axis=0)
        #print('medians:', medians)
        return np.mean(medians)
        
        
    
    def add(self, item):
        for i in range(self.nHash):
            for j in range(self.nGroups):    
                hash_seq = i * self.nHash + j
                # print('Hash number: ', hash_seq)
                bitstring = self.fmHash(item, hash_seq)
                tailLength = self.tail_length(bitstring)
                if tailLength > self.maxTailLength[i,j]:
                    self.maxTailLength[i,j] = tailLength

## Testing the algorithm

We will test our algorithm against actual number of distinct elements using a dictionary (which will be slow if the number of distinct elements is very large). This is surely not the ideal way to use this algorithm (when exact counting is possible). Ideally, one should use it for estimating the number of distinct elements in gigabytes of data (you may want to try that with a huge text corpus). 

In [ ]:
# dictionary for counting actual number of distinct elements
distinctElements = {}

# Our FM-estimator
fm = FlajoletMartinEstimator(64,5,6)

for i in range(10000):
    a = str(np.random.randint(100000))
    fm.add(a)
    distinctElements[a] = 1
# print(fm.maxTailLength)
print("Estimate 1: ", fm.estimate1())
print("Estimate 2: ", fm.estimate2())
print("Actual: ", len(distinctElements))